### Commands

In [1]:
from firecloud import api as firecloud_api
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
# res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'CumPoN21')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [4]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

---

---

In [15]:
### Create PoN from normals of only one batch
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals([all_paths], -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

,sample_id,individual_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code
0,AA33-Normal-SM-D4L4D,AA33,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AA33N_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59KF,SM-D4L4D,Normal,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA33N_3D_p5,NaN,NDNA
1,AA34-Tumor-SM-D4L25,AA34,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AA34T_3D_p4,TSCA Rapid Cancer Detection Panel v2,SM-D59I2,SM-D4L25,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA34T_3D_p4,Metastatic,TM
2,AA36-Tumor-SM-D4L2H,AA36,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AA36T_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59IE,SM-D4L2H,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA36T_3D_p5,Metastatic,TM
3,AA39-Tumor-SM-D4L3Q,AA39,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AA39T_OPAC_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59JQ,SM-D4L3Q,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA39T_OPAC_3D_p5,Primary,TP
4,AA40-Tumor-SM-D4L42,AA40,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AA40T_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59K3,SM-D4L42,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA40T_3D_p5,Primary,TP
5,AB044-Tumor-SM-D4L2M,AB044,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,AB044T_primary,TSCA Rapid Cancer Detection Panel v2,SM-D59IJ,SM-D4L2M,Tumor,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,AB044T_primary,Primary,TP
6,ANGMCSS-Tumor-SM-D4L2G,ANGMCSS,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,ANGMCSS_DMEMF12_p3,TSCA Rapid Cancer Detection Panel v2,SM-D59ID,SM-D4L2G,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",ANGMCSS_DMEMF12_p3,Primary,TP
7,BT1008-Tumor-SM-D4L45,BT1008,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,BT1008_primary,TSCA Rapid Cancer Detection Panel v2,SM-D59K7,SM-D4L45,Tumor,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,BT1008_primary,Primary,TP
8,BT922-Tumor-SM-D4L1W,BT922,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,BT922_CM_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59HT,SM-D4L1W,Tumor,PCR,Cells:Pellet frozen,Cells:Growing,BT922_CM_p5,Metastatic,TM
9,BT923-Tumor-SM-D4L3W,BT923,/xchip/clf/seq_data/processed_for_fh/tsca1213_...,BT923_NEURO_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59JX,SM-D4L3W,Tumor,PCR,Cells:Pellet frozen,Cells:Growing,BT923_NEURO_p5,Primary,TP
